# This jupyter notebook focuses on advanced parametrs of model config:
1. [microbatch](#microbatch)
2. [device](#device)
3. [train_steps](#train_steps)

---

Import Libraries.
Specify which GPU(s) to be used. More about it in [CUDA documentation](https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#env-vars).

In [1]:
%load_ext autoreload
%autoreload 2
import os
import sys
import warnings
os.environ['CUDA_DEVICE_ORDER']='PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES']='4,5,6,7' # specify which GPU(s) to be used

import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)

sys.path.append('../..')
from batchflow import Pipeline, B, C, V, D
from batchflow.opensets import MNIST
from batchflow.models.tf import VGG7

In [2]:
BATCH_SIZE = 64

---

# Create a dataset

Downloading MNIST. You now about MNIST from [here](./02_pipeline_operations.ipynb).

In [3]:
dataset = MNIST(bar=True)

100%|██████████| 8/8 [00:01<00:00,  2.93it/s]


---

# Define a pipeline config

Default pipeline config. You now about it from [this notebook](./03_ready_to_use_model_tf.ipynb).

In [4]:
config = dict(model=VGG7)

---

# Define a model config

Default model config.

Additional advanced options will be added to model_config.

In [5]:
model_config = {'inputs': {'images/shape': (B('image_shape')),
                           'labels': {'classes': D('num_classes'),
                                      'transform': 'ohe'}},
                'initial_block': {'inputs': 'images'}}

---
<a id='microbatch'></a>
### Microbatch 

**Microbatching** allows to process given data sequentially, accumulating gradients from microbatches and applying them once in the end. The size of the microbatch can be specified in two places; the value that was specified last will be used.

<font color='red'>**Microbatch size must be a divisor of the batch size!**</font>

Set size of microbatch 32.

In [6]:
model_config.update({'microbatch': 32})

Now, if we run the pipeline, the model will receive batches with size 32 not BATCH_SIZE.

---
<a id='device'></a>
### device

You may need more than one GPU if model training time consumes a significant fraction of execution pipeline time.

Parameter **device** allows train model on multiple GPU (Сreates a copy of model on each selected GPU).
Initialization of large model on a large number of GPU may take some time (minuts or tens of minutes)! 

Parameter **device** can be either string or sequence of strings.

Example:
```python
'device': 'GPU:0'            # Used only GPU:0
'device': ['GPU:0', 'GPU:1'] # Used GPU:0 and GPU:1
'device': 'GPU:*'            # Used all avalible GPU
```
<font color='red'>**Number of devices must be a divisor of the batch size! (If microbathing ~~batch size~~ microbatch size)**</font>

---
<a id='train_steps'></a>
### train_steps

*scope* – subset of weights to optimize during training. Can be either string or sequence of strings.
Value ```''``` is reserved for optimizing all trainable variables. Putting ```-``` sign before name stands for complement: optimize everything but the passed scope. Scope can be choosen from masks of the path to the model weights tensors.

Also can be used list of scopes:
```python
'scope': ['block/group-0/some_layer', 'block/some_group']
```

**train_steps** – configuration of different training procedures. It allows to optimize parametrs of selected scope using selected optimizer, loss, decay.  Watch [FreezeOut](https://arxiv.org/abs/1706.04983) to find out what it is.

Give name (key of dict) of train_step as you wish. Choose optimizer, scope of weights of your neural network,
and learning rate decay config ([Avalible losses and decays](https://analysiscenter.github.io/batchflow/api/batchflow.models.tf.base.html)). For example:


```python
'train_steps': {'name_of_train_step': {'optimizer': 'Adam', 
                                       'scope': 'block/group/layer', 
                                       'decay': lr_decay_config, 
                                       'loss': 'mse'}}
```

Parameter **train_mode** used in train_model to select *train_step*. 
To fetch loss according to selected *train_step* use ```fetches='loss_name_of_train_step'```.


When used:
```python 
train_mode='name_of_train_step'
```
selected ```name_of_train_step``` train_step with selected config inside: optimizer - ```Adam```,
learninig rate decay preset in ```lr_decay_config```, loss - ```mse```, and optimize only weights in ```block/group/layer```.  When the model is training loss according ```name_of_train_step``` *train_step* can be fetched as ```fetches='loss_name_of_train_step'```.

#### train_steps works only on tensorflow models!

Add *train_steps* in model_config.

In [7]:
lr_decay_config = ('exp', {'learning_rate': 0.005,
                           'decay_steps': 100,
                           'decay_rate': 0.96})

model_config.update({'train_steps': 
                     {'all': {'optimizer': 'Adam'},
                      'all_with_decay': {'optimizer': 'Adam', 'decay': lr_decay_config},
                      'custom': {'optimizer': 'RMSProp', 'scope': '-body/block-0', 'decay': lr_decay_config},
                      'part_head': {'use': 'all', 'scope': 'head/layer-2', 'loss': 'mse'}
                      }})

Also scope contain all trainable variables if it is not set (As in the case ```'all'``` or ```'all_with_decay'```).

Optimizer and decay together may be reused by another *train_step*. Use key ```'use'``` and name of *train_step* to do that (As in the case ```'part_head'```).

In [8]:
model_config.update({'device': 'CPU:*'})

To train model used all avalible GPU(s).

---

# Create a template pipeline

Now we use ```train_mode='all'``` to select train_step ```'all'```. You could see such pipeline and all that comes next in [3 tutorial](./03_ready_to_use_model_tf.ipynb).

In [9]:
train_template = (Pipeline(config=config)
                  .to_array()
                  .train_model(name='conv_nn',
                               use_lock=True,
                               fetches='loss_all', 
                               images=B('images'), 
                               labels=B('labels'),
                               save_to=V('current_loss'), 
                               train_mode='all')
                  .update_variable('loss_history', V('current_loss'), mode='a'))

(train_template.before
 .init_variable('loss_history', init_on_each_run=list)
 .init_variable('current_loss')
 .init_model(mode='dynamic',
             model_class=C('model'),
             name='conv_nn',
             config=model_config))

---
# Train the model

Apply a dataset to a template pipeline to create a runnable pipeline:

In [10]:
train_pipeline = train_template << dataset.train
train_pipeline.run(BATCH_SIZE, shuffle=True, n_epochs=1, bar=True, drop_last=True)

100%|██████████| 937/937 [15:39<00:00,  1.04it/s]


---
# Test the model

In [11]:
test_pipeline = (Pipeline()
                 .to_array()
                 .predict_model(name='conv_nn',
                                fetches='predictions', 
                                images=B('images'), 
                                save_to=V('predictions'))
                 .gather_metrics('class', targets=B('labels'), predictions=V('predictions'),
                                 fmt='logits', axis=-1, save_to=V('metrics', mode='a')))

(test_pipeline.before
 .init_variable('predictions') 
 .init_variable('metrics', init_on_each_run=None)
 .import_model(model='conv_nn',
               pipeline=train_pipeline))

In [12]:
test_pipeline = (test_pipeline << dataset.test)
test_pipeline.run(BATCH_SIZE, shuffle=True, n_epochs=1, bar=True)

 99%|█████████▉| 156/157 [00:40<00:00,  4.00it/s]


Let's get the accumulated [metrics information](https://analysiscenter.github.io/batchflow/intro/models.html#model-metrics)

In [13]:
metrics = test_pipeline.get_variable('metrics')
metrics.evaluate('accuracy')

0.9912420382165605